In [6]:
from igraph import *
import random
import csv

In [7]:
g = Graph()

g1 = Graph.GRG(100, 0.2)

g2 = Graph.Tree(127, 2)

In [8]:
teams = 20
players_per_team = 20
min_stayed = 0.7 * players_per_team
transfer_windows = 10

In [9]:
#Generate team_labels
team_labels = []
for i in range(0,teams):
    lst = [*range(i*players_per_team, (i+1)*players_per_team)]
    team_labels.append(lst)
    
#Generate graph
test_graph = Graph()
test_graph.add_vertices(teams*players_per_team)
for t in range(0,teams):
    for i in range(0,players_per_team):
        for j in range(i,players_per_team-1):
            test_graph.add_edges([(i + t * players_per_team ,j+ 1 + t * players_per_team)])
            
          
#Make transfers
for times in range(0,transfer_windows):
    for i in range(0,teams-1):
        players_chosen = int((len(team_labels[i]) - 14) / 2)
        for j in range(0,players_chosen):
            player_to_transfer = (team_labels[i][random.randint(0, len(team_labels[i]) - 1)])
            team_labels[i].remove(player_to_transfer)
            new_team = random.randint(0,teams-1)
            if new_team == 1:
                new_team += 1
                new_team %= teams
            team_labels[new_team].append(player_to_transfer)
            for k in team_labels[new_team]:
                test_graph.add_edges([(player_to_transfer,k)])



In [1]:
print(test_graph)

In [76]:
def GirvanNewman(graph,result):
    while(graph.ecount()>0):
        edge_betweenness = graph.edge_betweenness(False, None, None)
        max_eb = max(edge_betweenness)
        
        edgesToDelete = [graph.es[idx].tuple for idx, eb in enumerate(edge_betweenness) if (eb == max_eb and eb == max_eb)]
        result.append(edgesToDelete)
        graph.delete_edges(edgesToDelete)

In [2]:
res = list()
GirvanNewman(test_graph,res)
res

In [65]:
def CONGARandom(g,n):
    
    iteration=0
    new_vertex_counter = g.vcount()
    table = [[-1]]*100*g.vcount()
    
    while(iteration<n and g.ecount() > 0):
        edge_betweenness = g.edge_betweenness(False, None, None)
        max_eb = max(edge_betweenness)
        vertex_betweenness = g.betweenness(None, False, None, None, True)
        max_vb = max(vertex_betweenness)

        if(max_eb > max_vb):
            edgesToDelete = [g.es[idx].tuple for idx, eb in enumerate(edge_betweenness) if (eb <= max_eb+0.1 and eb >= max_eb-0.1)]
            g.delete_edges(edgesToDelete)
            
        else:
            vertexToCopy = vertex_betweenness.index(max_vb);
            incident = g.incident(vertexToCopy, mode=ALL)
            g.add_vertices(str(new_vertex_counter))
            table[vertexToCopy].append(new_vertex_counter)
            delete_edge_iterator=0
            length = len(incident)/2
            edges_to_delete = [];
            while(delete_edge_iterator < length):
                source = g.es[incident[delete_edge_iterator]].source
                target = g.es[incident[delete_edge_iterator]].target
                if(target == vertexToCopy):
                    g.add_edges([(source, new_vertex_counter)])
                else:
                    g.add_edges([(new_vertex_counter, target)])
                edges_to_delete.append(incident[delete_edge_iterator])
                delete_edge_iterator+=1
            new_vertex_counter+=1
            g.delete_edges(edges_to_delete)
            incident2 = g.incident(vertexToCopy, mode=ALL)
        iteration+=1
        print(g.ecount())
    return table

In [3]:
tab = CONGARandom(test_graph,10)
tab

In [62]:
def CONGARandom1(g,n):
    
    iteration=0
    new_vertex_counter = g.vcount()
    table = [[-1]]*100*g.vcount()
    
    while(iteration<n and g.ecount() > 0):
        print(' ')
        print(iteration)
        edge_betweenness = g.edge_betweenness(directed=False, cutoff=None, weights=None)
        max_eb = max(edge_betweenness)
        print(max_eb)
        vertex_betweenness = g.betweenness(vertices=None, directed=False, cutoff=None, weights=None, nobigint=True)
        max_vb = max(vertex_betweenness)
        print(max_vb)
        if(max_eb > max_vb):
            edgesToDelete = [g.es[idx].tuple for idx, eb in enumerate(edge_betweenness) if (eb <= max_eb+0.1 and eb >= max_eb-0.1)]
            #print(edgesToDelete)
            g.delete_edges(edgesToDelete)
        else:
            vertexToCopy = vertex_betweenness.index(max_vb);
            #print(vertexToCopy)
            incident = g.incident(vertexToCopy, mode=ALL)
            #print(len(incident))
            g.add_vertices(str(new_vertex_counter))
            table[vertexToCopy].append(new_vertex_counter)
            delete_edge_iterator=0
            length = len(incident)/2
            #print(incident)
            edges_to_delete = [];
            while(delete_edge_iterator < length):
                #g.delete_edges(incident[delete_edge_iterator])
                source = g.es[incident[delete_edge_iterator]].source
                target = g.es[incident[delete_edge_iterator]].target
                #print([source, target])
                if(target == vertexToCopy):
                    #print([(source, new_vertex_counter)])
                    g.add_edges([(source, new_vertex_counter)])
                else:
                    #print([(new_vertex_counter, target)])
                    g.add_edges([(new_vertex_counter, target)])
                edges_to_delete.append(incident[delete_edge_iterator])
                delete_edge_iterator+=1
            new_vertex_counter+=1
            #print(edges_to_delete)
            g.delete_edges(edges_to_delete)
            incident2 = g.incident(vertexToCopy, mode=ALL)
            #print(len(incident2))
            #print(incident2)
        iteration+=1
        #print(table)
        print(g.ecount())
        #print(g.vcount())
    return table

In [4]:
tab = CONGARandom1(test_graph,5)
tab

In [4]:
import csv
import collections
import random
from igraph import *

edgesFilename = "../../scrapers/La-Liga-StatsEdges.csv"
#edgesFilename = "genedges"

def delete_comments(csvfile):
    for row in csvfile:
        raw = row.split('#')[0].strip()
        if raw:
            yield raw

def read_graph( edges_file):
    print('reading from file')
    players = []
    edges = []
   

    with open(edges_file , 'r') as csvfile:
        csvreader = csv.reader(delete_comments(csvfile), delimiter=',')
        edges = list(csvreader)

    print('edges setup')
    edgevertices = list([edge[0] for edge in edges] + [edge[1] for edge in edges])

    print('edgevertices')
    edgevertices = list(dict.fromkeys(edgevertices))

    print('vertices')
    #vertices = list([player for player in players if player[0] in edgevertices])

    
    print('inverse_indicies')
    inverse_indicies = dict(zip(edgevertices, range(len(edgevertices))))

    g = Graph()

    g.add_vertices(len(edgevertices))
    print('no vertices   ' + str(len(edgevertices)))
    g.vs["id"] = [vertex for vertex in edgevertices]
    g.vs["label"] = g.vs["id"]
    g.vs["vertex_size"] = 8
    g.vs["newlpalabel"] = 0
    g.es["weight"] = 2
    g.vs["vertex_color"] = 0

    print("setting colors")
    for vert in g.vs:
        vert["vertex_color"] = f'rgb({random.randint(0, 255)}, {random.randint(0, 255)}, {random.randint(0, 255)})'
    g.vs["lpalabel"] = list(zip(range(len(edgevertices)), g.vs["vertex_color"]))

    print("adding edges")
    try:
        for edge in edges:
            #print(str(edge[0]) + " " + str(edge[1]))
            #edge1 = edge[0]
            #edge2 = edge[1]
            #i = i + 1
            g.add_edge(inverse_indicies[edge[0]], inverse_indicies[edge[1]])
    except:
        print("Error occured during graph creating")
        quit()

    g.es["weigth"] = 1
    print("graph created")
    return g

def create_dictionary(vertices):
    dictionary = dict(zip(vertices, range(len(vertices))))
    return dictionary

g = read_graph(edgesFilename)

reading from file
edges setup
edgevertices
vertices
inverse_indicies
no vertices   3701
setting colors
adding edges
graph created


In [ ]:
import igraph
igraph.Graph.simplify(g)
g

In [5]:
tab = GirvanNewman(g,tab)

In [6]:
tab